In [3]:
!pip install interruptingcow
!pip install whois
!pip install pyquery

In [4]:
from tqdm import tqdm
from interruptingcow import timeout
import os
import whois
from datetime import datetime, timezone
import math
import pandas as pd
import numpy as np
from pyquery import PyQuery
from requests import get

In [5]:
class UrlFeaturizer(object):
    def __init__(self, url):
        self.url = url 
        self.domain = url.split('//')[-1].split('/')[0]
        self.today = datetime.now().replace(tzinfo=None)

        try:
            self.whois = whois.query(self.domain).__dict__
        except:
            self.whois = None

        try:
            self.response = get(self.url)
            self.pq = PyQuery(self.response.text)
        except:
            self.response = None
            self.pq = None

    ## URL string Features

    def entropy(self):
        string = self.url.strip()
        prob = [float(string.count(c)) / len(string) for c in dict.fromkeys(list(string))]
        entropy = sum([(p * math.log(p) / math.log(2.0)) for p in prob])
        return entropy

    def ip(self):
        string = self.url
        flag = False
        if ("." in string):
            elements_array = string.strip().split(".")
            if(len(elements_array) == 4):
                for i in elements_array:
                    if (i.isnumeric() and int(i)>=0 and int(i)<=255):
                        flag=True
                    else:
                        flag=False
                        break
        if flag:
            return 1 
        else:
            return 0

    def numDigits(self):
        digits = [i for i in self.url if i.isdigit()]
        return len(digits)

    def urlLength(self):
        return len(self.url)

    def numParameters(self):
        params = self.url.split('&')
        return len(params) - 1

    def numFragments(self):
        fragments = self.url.split('#')
        return len(fragments) - 1

    def numSubDomains(self):
        subdomains = self.url.split('http')[-1].split('//')[-1].split('/')
        return len(subdomains)-1

    def domainExtension(self):
        ext = self.url.split('.')[-1].split('/')[0]
        return ext

    ## URL domain features
    def hasHttp(self):
        return 'http:' in self.url

    def hasHttps(self):
        return 'https:' in self.url

    def daysSinceRegistration(self):
        if self.whois and self.whois['creation_date']:
            diff = self.today - self.whois['creation_date'].replace(tzinfo=None)
            diff = str(diff).split(' days')[0]
            return diff
        else:
            return 0

    def daysSinceExpiration(self):
        if self.whois and self.whois['expiration_date']:
            diff = self.whois['expiration_date'].replace(tzinfo=None) - self.today
            diff = str(diff).split(' days')[0]
            return diff
        else:
            return 0
    
     ## URL Page Features
    def bodyLength(self):
        if self.pq is not None:
            return len(self.pq('html').text()) if self.urlIsLive else 0
        else:
            return 0

    def numTitles(self):
        if self.pq is not None:
            titles = ['h{}'.format(i) for i in range(7)]
            titles = [self.pq(i).items() for i in titles]
            return len([item for s in titles for item in s])
        else:
            return 0

    def numImages(self):
        if self.pq is not None:
            return len([i for i in self.pq('img').items()])
        else:
            return 0

    def numLinks(self):
        if self.pq is not None:
            return len([i for i in self.pq('a').items()])
        else:
            return 0

    def scriptLength(self):
        if self.pq is not None:
            return len(self.pq('script').text())
        else:
            return 0

    def specialCharacters(self):
        if self.pq is not None:
            bodyText = self.pq('html').text()
            schars = [i for i in bodyText if not i.isdigit() and not i.isalpha()]
            return len(schars)
        else:
            return 0

    def scriptToSpecialCharsRatio(self):
        v = self.specialCharacters()
        if self.pq is not None and v!=0:
            sscr = self.scriptLength()/v
        else:
            sscr = 0
        return sscr

    def scriptTobodyRatio(self):
        v = self.bodyLength()
        if self.pq is not None and v!=0:
            sbr = self.scriptLength()/v
        else:
            sbr = 0
        return sbr

    def bodyToSpecialCharRatio(self):
        v = self.bodyLength()
        if self.pq is not None and v!=0:
            bscr = self.specialCharacters()/v
        else:
            bscr = 0
        return bscr

    def urlIsLive(self):
        return self.response == 200

    def run(self):
        data = {}
        data['entropy'] = self.entropy()
        data['numDigits'] = self.numDigits()
        data['urlLength'] = self.urlLength()
        data['numParams'] = self.numParameters()
        data['hasHttp'] = self.hasHttp()
        data['hasHttps'] = self.hasHttps()
        data['urlIsLive'] = self.urlIsLive()
        data['bodyLength'] = self.bodyLength()
        data['numTitles'] = self.numTitles()
        data['numImages'] = self.numImages()
        data['numLinks'] = self.numLinks()
        data['scriptLength'] = self.scriptLength()
        data['specialChars'] = self.specialCharacters()
        data['ext'] = self.domainExtension()
        data['dsr'] = self.daysSinceRegistration()
        data['dse'] = self.daysSinceExpiration()
        data['sscr'] = self.scriptToSpecialCharsRatio()
        data['sbr'] = self.scriptTobodyRatio()
        data['bscr'] = self.bodyToSpecialCharRatio()
        data['num_%20'] = self.url.count("%20")
        data['num_@'] = self.url.count("@")
        data['has_ip'] = self.ip()
    
        return data

In [10]:
#path1= "C:\Users\ADI\Desktop\Major Project\DefacementSitesURLFiltered.csv"
#path2="/content/drive/My Drive/Major_Project/FinalDataset/URL/spam_dataset.csv"
#path3="/content/drive/My Drive/Major_Project/FinalDataset/URL/Malware_dataset.csv"
#path4="/content/drive/My Drive/Major_Project/FinalDataset/URL/Benign_list_big_final.csv"
defacement = pd.read_csv("DefacementSitesURLFiltered.csv")
spam = pd.read_csv("spam_dataset.csv")
malware = pd.read_csv("Malware_dataset1.csv")
benign = pd.read_csv("Benign_list_big_final.csv")

In [11]:
defacement.describe()

,http://www.sinduscongoias.com.br/index.html
count,96456
unique,95307
top,http://www.tandemimmobilier.fr/index.php?optio...
freq,3


In [12]:
# Define systematic sampling function
def systematic_sampling(df, step):
    
    indexes = np.arange(0,len(df),step=step)
    systematic_sample = df.iloc[indexes]
    return systematic_sample
    
# Obtain a systematic sample and save it in a new variable
defacement = systematic_sampling(defacement, 20)
defacement.reset_index(inplace = True)
defacement=defacement.iloc[: , 1:]
defacement

,http://www.sinduscongoias.com.br/index.html
0,http://www.sinduscongoias.com.br/index.php/ins...
1,http://www.sinduscongoias.com.br/index.php/com...
2,http://www.pontoprofissional.com.br/portal/ind...
3,http://www.coleyglesias.com/index.php?option=c...
4,http://www.coleyglesias.com/index.php?option=c...
...,...
4818,http://www.platingroup.az/online-test/index.html
4819,http://www.platingroup.az/en/telim-teqvimi/day...
4820,http://www.platingroup.az/en/telim-teqvimi/day...
4821,http://www.platingroup.az/en/component/jquarks...


In [13]:
benign.describe()

,http://1337x.to/torrent/1048648/American-Sniper-2014-MD-iTALiAN-DVDSCR-X264-BST-MT/
count,35377
unique,35377
top,http://extratorrent.cc/torrent_download/418903...
freq,1


In [14]:
benign = systematic_sampling(benign, 8)
benign.reset_index(inplace = True)
benign=benign.iloc[: , 1:]
benign

,http://1337x.to/torrent/1048648/American-Sniper-2014-MD-iTALiAN-DVDSCR-X264-BST-MT/
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://abc.go.com/shows/general-hospital/episo...
2,http://allegro.pl/sexy-g-string-stringi-z-kora...
3,http://allrecipes.com/Recipe/Fluffy-Pancakes-2...
4,http://anandtech.com/show/9199/lg-announces-th...
...,...
4418,http://babal.net/books/view/2/%D9%85%D8%B9%D9%...
4419,http://babal.net/books/view/76/%D9%85%D8%B9%D9...
4420,https://lastpass.com/signup2.php?ac=1&from_uri...
4421,https://lastpass.com/signup2.php?ac=1&from_uri...


In [15]:
spam.describe()

,http://astore.amazon.co.uk/allezvinsfrenchr/detail/1904010202/026-8324244-9330038
count,11999
unique,11920
top,http://ancestry.co.uk/search/rectype/alldblist...
freq,67


In [16]:
spam = systematic_sampling(spam, 3)
spam.reset_index(inplace = True)
spam=spam.iloc[: , 1:]
spam

,http://astore.amazon.co.uk/allezvinsfrenchr/detail/1904010202/026-8324244-9330038
0,http://archive.salisburyjournal.co.uk/2007/3/6...
1,http://acard4u.co.uk/product_reviews.php?cPath...
2,http://archive.yorkpress.co.uk/2007/5/2/356167...
3,http://abyssinica.co.uk/_vti_bin/shtml.dll/_ta...
4,http://applerugs.co.uk/rugs/product_reviews.ph...
...,...
3995,http://abdn.ac.uk/registry/courses/course.php?...
3996,http://aolshop.memberoffers.co.uk/csp/smp/aol/...
3997,http://ancestry.co.uk/search/db.aspx?dbid=13208
3998,http://archive.salisburyjournal.co.uk/2001/3/7/


In [17]:
malware.describe()

,http://gzzax.livechatvalue.com/chat/chatClient/chatbox.jsp?companyID=11806&enterurl=file%3A%2F%2F%2FC%3A%2FUsers%2Famelia%2FDesktop%2F%25E7%25BD%2591%25E4%25B8%258A%25E8%25B5%258C%25E5%259C%25BA.html&pagetitle=&pagereferrer=&firstEnterUrl=file%3A%2F%2F%2FC%3A%2FUsers%2Famelia%2FDesktop%2F%25E7%25BD%2591%25E4%25B8%258A%25E8%25B5%258C%25E5%259C%25BA.html&lan=zh&tm=1420533810210
count,2691
unique,2691
top,http://amos.im.alisoft.com/online.aw?v=2&uid=%...
freq,1


In [18]:
benign.to_csv("Benign_dataset.csv",mode = 'w', index=False)
defacement.to_csv("Defacement_dataset.csv",mode = 'w', index=False)
spam.to_csv("Spam_dataset.csv",mode = 'w', index=False)

In [19]:
l = ['Defacement_dataset.csv','Malware_dataset1.csv','Spam_dataset.csv','Benign_dataset.csv']

In [21]:
emp = UrlFeaturizer("").run().keys()

A = pd.DataFrame(columns = emp)
t=[]
for j in l:
    print(j)
    d=pd.read_csv(j,header=None).to_numpy().flatten()
    for i in tqdm(d):  
        temp=UrlFeaturizer(i).run()
        temp["File"]=j.split(".")[0]
        t.append(temp) 
A=A.append(t)
os.chdir('../')
A.to_csv("Features(4k).csv")

  0%|          | 0/4824 [00:00<?, ?it/s]

Defacement_dataset.csv
downloading dependencies
copy \\live.sysinternals.com\tools\whois.exe c:\Users\ADI\Desktop\Major Project


  0%|          | 1/4824 [00:29<39:54:42, 29.79s/it]

downloading dependencies
copy \\live.sysinternals.com\tools\whois.exe c:\Users\ADI\Desktop\Major Project


  0%|          | 2/4824 [00:48<31:01:12, 23.16s/it]

...

downloading dependencies
copy \\live.sysinternals.com\tools\whois.exe c:\Users\ADI\Desktop\Major Project


 11%|█         | 512/4824 [31:12<16:09:37, 13.49s/it]

downloading dependencies
copy \\live.sysinternals.com\tools\whois.exe c:\Users\ADI\Desktop\Major Project


 11%|█         | 513/4824 [31:24<15:38:55, 13.07s/it]

downloading dependencies
copy \\live.sysinternals.com\tools\whois.exe c:\Users\ADI\Desktop\Major Project
